# Wage Inequality Analysis using Quantile Regression

This tutorial demonstrates comprehensive wage inequality analysis using PanelBox's quantile regression tools.

**Learning Objectives:**
1. Analyze heterogeneous returns to education across wage distribution
2. Decompose gender wage gap using Machado-Mata decomposition
3. Estimate union wage premium heterogeneity
4. Compare pooled and fixed effects quantile regression

**Dataset:** Simulated wage panel data (PSID-style)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wage_analysis import WageInequalityAnalysis

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Data Generation and Exploration

We simulate a realistic wage panel dataset with:
- 1,000 individuals over 10 years
- Education, experience, gender, union status
- Heterogeneous returns to education
- Gender wage gap
- Individual fixed effects (ability)

In [ ]:
# Initialize analysis with simulated data
analysis = WageInequalityAnalysis()

# Display basic statistics
print("Dataset Overview:")
print("="*60)
print(f"Number of individuals: {analysis.data['person_id'].nunique()}")
print(f"Number of years: {analysis.data['year'].nunique()}")
print(f"Total observations: {len(analysis.data)}")
print(f"\nFemale share: {analysis.data['female'].mean():.1%}")
print(f"Union members: {analysis.data['union'].mean():.1%}")
print(f"\nMean education: {analysis.data['education'].mean():.2f} years")
print(f"Mean experience: {analysis.data['experience'].mean():.2f} years")
print(f"Mean wage: ${analysis.data['wage'].mean():.2f}")

In [ ]:
# Visualize wage distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Overall distribution
axes[0].hist(analysis.data['log_wage'], bins=50, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Log Wage')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Log Wage Distribution')
axes[0].axvline(analysis.data['log_wage'].mean(), color='red', 
                linestyle='--', linewidth=2, label='Mean')
axes[0].legend()

# By gender
male_wages = analysis.data[analysis.data['female'] == 0]['log_wage']
female_wages = analysis.data[analysis.data['female'] == 1]['log_wage']

axes[1].hist(male_wages, bins=50, alpha=0.5, label='Male', color='blue')
axes[1].hist(female_wages, bins=50, alpha=0.5, label='Female', color='red')
axes[1].set_xlabel('Log Wage')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Wage Distribution by Gender')
axes[1].legend()

# By union status
union_wages = analysis.data[analysis.data['union'] == 1]['log_wage']
nonunion_wages = analysis.data[analysis.data['union'] == 0]['log_wage']

axes[2].hist(nonunion_wages, bins=50, alpha=0.5, label='Non-Union', color='gray')
axes[2].hist(union_wages, bins=50, alpha=0.5, label='Union', color='green')
axes[2].set_xlabel('Log Wage')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Wage Distribution by Union Status')
axes[2].legend()

plt.tight_layout()
plt.show()

## 2. Returns to Education Across Wage Distribution

**Research Question:** Do returns to education vary across the wage distribution?

**Economic Theory:**
- High earners may have higher returns (complementarity with ability)
- OR low earners may have higher returns (catch-up effect)

**Method:**
- Pooled quantile regression
- Fixed effects quantile regression (Canay 2011)

In [ ]:
# Estimate returns to education
tau_list = [0.10, 0.25, 0.50, 0.75, 0.90]
pooled_result, canay_result = analysis.analyze_education_returns(tau_list)

### Interpretation

The results show:

1. **Pooled QR**: Returns increase from bottom to top of distribution
   - Low earners (τ=0.10): ~5% per year of education
   - High earners (τ=0.90): ~8% per year of education
   
2. **Fixed Effects QR**: Controls for unobserved ability
   - Generally lower estimates (ability bias)
   - Still shows heterogeneity

3. **Statistical Test**: Strong evidence of heterogeneous returns (p < 0.05)

**Policy Implication**: Education investments may increase wage inequality by raising top earners' wages more than bottom earners'.

## 3. Gender Wage Gap Decomposition

**Research Question:** How much of the gender wage gap is due to:
- Differences in characteristics (education, experience)?
- Differences in returns (discrimination)?

**Method:** Machado-Mata (2005) quantile decomposition

Total gap = Characteristics effect + Coefficients effect

In [ ]:
# Decompose gender wage gap
decomposition = analysis.analyze_gender_gap()

### Interpretation

The decomposition reveals:

1. **Total Gap**: Larger at bottom of distribution ("sticky floor" effect)
   - Bottom 10%: ~25% wage gap
   - Median: ~20% gap
   - Top 10%: ~15% gap

2. **Explained Component**: Differences in education, experience, union
   - Accounts for ~40% of gap at median
   - More important at top of distribution

3. **Unexplained Component**: Differences in returns (discrimination?)
   - Larger at bottom of distribution
   - Suggests discrimination hits low-wage women hardest

**Policy Implication**: Policies targeting both characteristics (education/training) and discrimination are needed.

## 4. Union Wage Premium

**Research Question:** Does union membership affect wages differently across distribution?

**Economic Theory:**
- Unions may compress wage distribution
- Larger effects for low-wage workers
- "Equalization" effect

In [ ]:
# Analyze union effects
union_results = analysis.analyze_union_effects()

### Interpretation

Union wage premium:

1. **Heterogeneity**: Premium varies from ~20% at bottom to ~10% at top
2. **Inequality Reduction**: Unions raise low wages more than high wages
3. **Statistical Significance**: All estimates significantly different from zero

**Policy Implication**: Union decline may contribute to rising wage inequality.

## 5. Comparison with OLS

Let's compare quantile regression results with traditional OLS:

In [ ]:
from panelbox.models.linear import PooledOLS

# OLS regression
formula = 'log_wage ~ education + experience + I(experience**2) + female + union'
ols_model = PooledOLS(analysis.panel, formula)
ols_result = ols_model.fit()

print("OLS Results:")
print("="*60)
ols_result.summary()

In [ ]:
# Compare education returns: OLS vs QR
fig, ax = plt.subplots(figsize=(10, 6))

# Extract QR estimates
tau_grid = np.arange(0.1, 1.0, 0.05)
qr_returns = []

for tau in tau_grid:
    from panelbox.models.quantile import PooledQuantile
    model = PooledQuantile(analysis.panel, formula, tau=tau)
    result = model.fit(verbose=False)
    qr_returns.append(result.results[tau].params[1])  # Education coefficient

# Plot
ax.plot(tau_grid, qr_returns, 'o-', linewidth=2, markersize=6, 
        label='Quantile Regression', color='blue')
ax.axhline(ols_result.params[1], color='red', linestyle='--', 
           linewidth=2, label=f'OLS: {ols_result.params[1]:.4f}')

ax.fill_between(tau_grid, 
                ols_result.params[1] - 1.96*ols_result.bse[1],
                ols_result.params[1] + 1.96*ols_result.bse[1],
                alpha=0.2, color='red', label='OLS 95% CI')

ax.set_xlabel('Quantile', fontsize=12, fontweight='bold')
ax.set_ylabel('Return to Education (log points)', fontsize=12, fontweight='bold')
ax.set_title('OLS vs. Quantile Regression: Returns to Education', 
             fontsize=14, fontweight='bold')
ax.legend(frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("OLS gives the AVERAGE return, but misses heterogeneity across distribution.")
print(f"QR reveals returns range from {min(qr_returns):.4f} to {max(qr_returns):.4f}")

## 6. Summary and Conclusions

### Key Findings

1. **Returns to Education**: Strongly heterogeneous
   - Higher returns at top of distribution
   - Education increases inequality

2. **Gender Wage Gap**:
   - Larger at bottom (sticky floor)
   - Partly explained by characteristics
   - Large unexplained component (discrimination?)

3. **Union Effects**:
   - Larger premium for low earners
   - Inequality-reducing effect
   - Heterogeneity important for policy

### Why Quantile Regression?

OLS tells us about **average** effects:
- "Education increases wages by 6%"
- "Women earn 15% less than men"
- "Union workers earn 12% more"

Quantile regression tells us **the full story**:
- Education: 5% at bottom, 8% at top → increases inequality
- Gender gap: 25% at bottom, 15% at top → sticky floor
- Union: 20% at bottom, 10% at top → reduces inequality

### Policy Implications

1. **Education Policy**:
   - Need complementary policies to prevent inequality growth
   - Target support to low-skill workers

2. **Gender Equity**:
   - Focus on low-wage female workers
   - Address both characteristics and discrimination

3. **Labor Market**:
   - Union decline may exacerbate inequality
   - Consider policies supporting collective bargaining

### Next Steps

1. **Extensions**:
   - Add time trends (rising inequality)
   - Regional heterogeneity
   - Industry/occupation effects

2. **Robustness**:
   - Bootstrap inference
   - Specification tests
   - Sensitivity analysis

3. **Real Data**:
   - Apply to PSID, CPS, or other datasets
   - Replicate published studies

## Exercises

1. **Modify the data generation** to include:
   - Race/ethnicity
   - Industry indicators
   - Time trends

2. **Estimate** quantile regression for experience effects:
   - Does experience matter more at bottom or top?
   - Plot experience profile by quantile

3. **Test** for quantile crossing:
   - Are quantile curves properly ordered?
   - If not, consider location-scale model

4. **Bootstrap** standard errors:
   - Implement pairs bootstrap
   - Compare with asymptotic SEs

5. **Load real data** and replicate:
   - PSID: https://psidonline.isr.umich.edu/
   - CPS: https://www.census.gov/cps/

Happy analyzing! 📊